In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df4 = pd.read_csv("data/test/ad.csv")
df2 = pd.concat([df2, df4])
df2.sort_values('creative_id', inplace=True)
df2 = df2.drop_duplicates(subset=['creative_id'], keep='first')
df3 = pd.read_csv("data/train_preliminary/click_log.csv")
df5 = pd.read_csv("data/test/click_log.csv")
df3 = pd.concat([df3, df5])
df = pd.merge(df3, df2, left_on='creative_id', right_on='creative_id')
order = ['user_id', 'time', 'creative_id', 'click_times', 'ad_id', 'product_category', 'advertiser_id']
df = df[order]
df.sort_values(['user_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

Using TensorFlow backend.


,user_id,time,creative_id,click_times,ad_id,product_category,advertiser_id
0,1,20,877468,1,773445,5,29455
1,1,20,209778,1,188507,2,9702
2,1,20,821396,1,724607,5,7293
3,1,39,1683713,1,1458878,5,14668
4,1,40,122032,1,109959,2,11411
...,...,...,...,...,...,...,...
63668278,4000000,75,3596158,1,3096233,18,36668
63668279,4000000,75,3642395,1,3135640,2,18422
63668280,4000000,76,366858,1,331268,2,36890
63668281,4000000,76,3333680,1,2868147,2,32830


In [4]:
x_tmp = np.array(df['creative_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(5000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['creative_id_0', 'creative_id_1', 'creative_id_2', 'creative_id_3', 'creative_id_4'])
X = df[['user_id', 'time', 'click_times']]
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['ad_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(4000000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['ad_id_0', 'ad_id_1', 'ad_id_2', 'ad_id_3', 'ad_id_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['product_category'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
print(x_tmp.shape)
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(20, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['product_category_0', 'product_category_1', 
                                                   'product_category_2', 'product_category_3', 'product_category_4'])
X = pd.concat([X, x_tmp], axis=1)
x_tmp = np.array(df['advertiser_id'])
x_tmp = x_tmp.reshape((-1, 1))
x_tmp = np.vstack((x_tmp, np.zeros((17, 1))))
x_tmp = x_tmp.reshape((-1, 100))
model = Sequential()
model.add(Embedding(70000, 5, input_length=100))
model.compile(loss='MSE', optimizer='adam')
y_tmp = model.predict(x_tmp, verbose=1)
y_tmp = y_tmp.reshape((-1, 5))
x_tmp = pd.DataFrame(y_tmp[:63668283, :], columns=['advertiser_id_0', 'advertiser_id_1', 
                                                   'advertiser_id_2', 'advertiser_id_3', 'advertiser_id_4'])
X = pd.concat([X, x_tmp], axis=1)
X

(63668300, 1)
636683/636683 [==============================] - 7s 11us/step
(63668300, 1)
636683/636683 [==============================] - 7s 11us/step
(63668300, 1)
636683/636683 [==============================] - 5s 7us/step


,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,20,1,-0.028821,0.043193,-0.002658,0.001140,0.010003,0.038925,-0.036180,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.035843,0.025101,0.033642,0.032090,0.012748
1,1,20,1,0.034695,-0.045004,-0.013941,0.006413,-0.020706,0.023887,-0.008284,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.044284,0.034583,0.040172,-0.040260,0.037008
2,1,20,1,-0.039916,0.049294,-0.005943,-0.005456,0.031549,0.013881,0.016362,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.021723,-0.008469,0.002402,-0.017408,-0.045114
3,1,39,1,-0.011225,-0.021940,-0.004938,0.045410,-0.033770,-0.000121,0.036999,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.033206,0.032321,0.041743,0.008567,-0.034515
4,1,40,1,-0.036918,-0.028326,-0.020239,0.026644,-0.041207,-0.030335,-0.039965,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.042767,0.047667,-0.024554,0.025078,0.016677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63668278,4000000,75,1,-0.004134,-0.024644,-0.000858,-0.006052,-0.003291,-0.011134,0.047722,...,-0.040473,-0.047419,0.014103,0.013628,-0.038049,0.010730,0.016806,0.025698,-0.029075,-0.038612
63668279,4000000,75,1,0.039157,0.041443,0.010109,-0.047209,-0.017556,-0.036195,0.044410,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.043261,-0.046551,-0.023381,-0.036314,-0.024812
63668280,4000000,76,1,0.043290,-0.024770,-0.049238,-0.010419,-0.039468,0.039936,0.048198,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,0.021748,-0.049302,0.012116,0.013548,0.042955
63668281,4000000,76,1,-0.031379,-0.022752,0.040858,0.001736,-0.021258,-0.033271,0.016498,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.017255,-0.049853,0.020312,0.007352,0.034234


In [5]:
X['time'] = X['time'] / 91
X['click_times'] = X['click_times'] / 152
X

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,1,0.219780,0.006579,-0.028821,0.043193,-0.002658,0.001140,0.010003,0.038925,-0.036180,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.035843,0.025101,0.033642,0.032090,0.012748
1,1,0.219780,0.006579,0.034695,-0.045004,-0.013941,0.006413,-0.020706,0.023887,-0.008284,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.044284,0.034583,0.040172,-0.040260,0.037008
2,1,0.219780,0.006579,-0.039916,0.049294,-0.005943,-0.005456,0.031549,0.013881,0.016362,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.021723,-0.008469,0.002402,-0.017408,-0.045114
3,1,0.428571,0.006579,-0.011225,-0.021940,-0.004938,0.045410,-0.033770,-0.000121,0.036999,...,-0.045872,-0.009993,0.007380,-0.009590,-0.025814,0.033206,0.032321,0.041743,0.008567,-0.034515
4,1,0.439560,0.006579,-0.036918,-0.028326,-0.020239,0.026644,-0.041207,-0.030335,-0.039965,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.042767,0.047667,-0.024554,0.025078,0.016677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63668278,4000000,0.824176,0.006579,-0.004134,-0.024644,-0.000858,-0.006052,-0.003291,-0.011134,0.047722,...,-0.040473,-0.047419,0.014103,0.013628,-0.038049,0.010730,0.016806,0.025698,-0.029075,-0.038612
63668279,4000000,0.824176,0.006579,0.039157,0.041443,0.010109,-0.047209,-0.017556,-0.036195,0.044410,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.043261,-0.046551,-0.023381,-0.036314,-0.024812
63668280,4000000,0.835165,0.006579,0.043290,-0.024770,-0.049238,-0.010419,-0.039468,0.039936,0.048198,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,0.021748,-0.049302,0.012116,0.013548,0.042955
63668281,4000000,0.835165,0.006579,-0.031379,-0.022752,0.040858,0.001736,-0.021258,-0.033271,0.016498,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.017255,-0.049853,0.020312,0.007352,0.034234


In [20]:
X_tmp = X.drop_duplicates(subset=['user_id'], keep='last')
X_tmp

,user_id,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
12,1,0.835165,0.013158,0.033997,0.045049,-0.016414,0.024355,-0.024740,-0.015207,0.028529,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.027068,-0.017845,0.004358,-0.047574,0.026212
57,2,0.989011,0.006579,-0.010723,-0.030561,0.017420,-0.036160,-0.008851,-0.001338,0.040919,...,-0.017840,0.043122,-0.006909,-0.028332,0.021584,-0.013753,0.011932,0.015922,-0.040905,0.038012
87,3,0.967033,0.006579,0.040299,0.042704,0.030630,-0.046512,-0.043972,0.020371,0.044276,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.034765,0.031936,-0.045832,0.001606,0.044473
116,4,0.923077,0.006579,-0.036224,-0.025655,0.012739,0.040950,-0.040724,0.015613,-0.027575,...,0.010659,-0.012860,-0.030064,0.043291,-0.011096,-0.021639,-0.012094,-0.017739,-0.034829,0.006503
149,5,0.956044,0.006579,0.034988,0.016111,-0.041836,-0.033163,-0.011913,-0.016228,-0.000511,...,-0.040473,-0.047419,0.014103,0.013628,-0.038049,0.029661,0.029463,0.039109,-0.004388,0.030258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63668100,3999996,0.989011,0.006579,-0.019040,0.004410,0.034132,0.031106,-0.020280,-0.028845,0.017527,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.039736,0.028690,-0.020712,0.017929,-0.040041
63668120,3999997,0.978022,0.006579,0.021530,-0.045349,-0.037061,0.036535,0.000007,0.006662,-0.032655,...,-0.040473,-0.047419,0.014103,0.013628,-0.038049,0.025870,-0.015403,0.006779,0.019540,0.011401
63668139,3999998,0.989011,0.006579,0.001309,0.009657,0.048617,-0.046487,-0.029696,0.006540,0.022698,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.017253,0.007553,-0.032331,0.044208,-0.023852
63668219,3999999,1.000000,0.006579,0.003083,-0.037439,0.005695,0.047994,0.026860,0.010253,-0.041143,...,-0.006258,0.020920,-0.031235,-0.012834,0.008495,-0.014676,-0.001150,0.040997,-0.047410,-0.030317


In [21]:
X_tmp = X_tmp['user_id'].reset_index()

In [23]:
X_tmp = X_tmp['index']

In [32]:
X_tmp

0                12
1                57
2                87
3               116
4               149
             ...   
1899995    63668100
1899996    63668120
1899997    63668139
1899998    63668219
1899999    63668282
Name: index, Length: 1900000, dtype: int64

In [35]:
from keras.utils import Sequence
class DataGenerater(Sequence):
    def __init__(self, batch_size, X, total, start_id, y_test, x_index):
        self.X = X
        self.batch_size = batch_size
        self.total = total
        self.start_id = start_id
        self.y_test = np.array(y_test)
        self.x_index = x_index
    def _check_matrix(self, x):
        if x.shape[0] >= 30:
            return x.iloc[:30, :]
        else:
            tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
            tmp1 = pd.DataFrame(tmp1)
            tmp1.columns = x.columns
            return pd.concat([x, tmp1], axis=0)
    def __getitem__(self, index):
        a = self.start_id + self.batch_size * index
        b = self.start_id + self.batch_size * (index + 1) - 1
        if index == 0:
            Y = self.X[:self.x_index.iloc[b] + 1]
        else:
            Y = self.X[self.x_index.iloc[a - 1] + 1:self.x_index.iloc[b] + 1]
        Y = Y.groupby('user_id').apply(self._check_matrix)
        Y = Y.reset_index(drop=True)
        Y = Y.drop('user_id', axis = 1)
        return np.array(Y).reshape((-1, 30, 22)), self.y_test[self.batch_size * index:self.batch_size * (index + 1)]
    def __len__(self):
        return self.total // self.batch_size

In [36]:
from keras.layers.core import Masking
train_generator = DataGenerater(batch_size=16, X=X, total=900000, start_id=0, y_test=df1.iloc[:900000, 2:3], 
                               x_index=X_tmp.iloc[:900000])
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit_generator(train_generator, nb_epoch=2)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=2)`
  if __name__ == '__main__':


Epoch 1/2
10380/56250 [====>.........................] - ETA: 20:35 - loss: 0.2228

KeyboardInterrupt: 

In [ ]:
Y1_train = df1.iloc[:900000, 1:2]
Y1_train = pd.get_dummies(Y1_train['age'])
model2 = Sequential()
model2.add(Masking(mask_value=0., input_shape=(30, 22)))
model2.add(LSTM(32))
model2.add(Dense(30, activation='relu'))
model2.add(Dense(10, activation='softmax'))
model2.compile(loss='MSE', optimizer='adam')
model2.fit(X_train, Y1_train, nb_epoch=2)

In [ ]:
Y = model2.predict(X_train)
ans = 0
for i in range(900000):
    if np.argmax(Y[i]) == df1.iloc[i, 1] - 1:
        ans += 1
print('age accurency:', ans/900000)

In [ ]:
Y = model.predict(X_train)
Y = np.round(Y)
ans = 0
for i in range(900000):
    if Y[i] == Y2_train.iloc[i, 0]:
        ans += 1
print('gender accurency:', ans/900000)

In [ ]:
import csv
Y2_output = model.predict(X_test)
Y1_output = model2.predict(X_test)
Y1_output = np.argmax(Y1_output, axis=0) + 1
Y2_output = np.round(Y2_output)
with open('submission.csv', 'w', encoding='utf-8') as csvfile:
    fieldnames = ['user_id', 'predicted_age', 'predicted_gender']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(1000000):
        writer.writerow({'user_id': str(i + 3000001), 'predicted_age': str(Y1_output[i]),
                         'predicted_gender': str(Y2_output[i])})